In [1]:
import pandas as pd

df = pd.read_csv('glassdoor_jobs.csv')
df.head(5)

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Business Analyst,Employer Provided Salary:$72K - $98K,1Starr Enterprises is hiring a Business Analys...,-1.0,1Starr,Remote,-1,-1,-1,-1,-1,-1,-1,-1
1,1,Entry Level Data Conversion Analyst,$35K - $71K (Glassdoor est.),"As the Data Conversion Analyst, you will be co...",3.4,Reynolds and Reynolds\n3.4,"Dayton, OH",-1,1001 to 5000 Employees,1866,Company - Private,Computer Hardware & Software,Information Technology,$2 to $5 billion (USD),-1
2,2,Junior Data Analyst,Employer Provided Salary:$30K,About Avalanche\nAvalanche is transforming how...,-1.0,Avalanche,Remote,-1,-1,-1,-1,-1,-1,-1,-1
3,3,Senior Data Analyst with Visualization (No spo...,$63K - $149K (Glassdoor est.),Senior Data Analyst with Visualization (No spo...,3.7,HCL Technologies\n3.7,"Burlingame, CA",-1,10000+ Employees,1991,Company - Public,Computer Hardware & Software,Information Technology,$5 to $10 billion (USD),-1
4,4,Data Analyst,$43K - $88K (Glassdoor est.),Dematic is looking for a Data Analyst that wil...,3.8,Dematic\n3.8,"Atlanta, GA",-1,5001 to 10000 Employees,1819,Subsidiary or Business Segment,Logistics & Supply Chain,Transportation & Logistics,Unknown / Non-Applicable,-1


In [2]:
df.shape

(1000, 15)

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              1000 non-null   int64  
 1   Job Title          1000 non-null   object 
 2   Salary Estimate    1000 non-null   object 
 3   Job Description    1000 non-null   object 
 4   Rating             1000 non-null   float64
 5   Company Name       1000 non-null   object 
 6   Location           1000 non-null   object 
 7   Headquarters       1000 non-null   int64  
 8   Size               1000 non-null   object 
 9   Founded            1000 non-null   object 
 10  Type of ownership  1000 non-null   object 
 11  Industry           1000 non-null   object 
 12  Sector             1000 non-null   object 
 13  Revenue            1000 non-null   object 
 14  Competitors        1000 non-null   int64  
dtypes: float64(1), int64(3), object(11)
memory usage: 117.3+ KB


In [4]:
# List of features 

df.columns

Index(['index', 'Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

In [5]:
# Delete columns containing only -1 values (NaN values)

del df['Headquarters']
del df['Competitors']


In [6]:
for feature in df.columns:
    if df[feature].dtypes == 'O':
        df = df[df[feature] != '-1']
    else:
        df = df[df[feature] != -1 ]
        

In [7]:
df.shape

(527, 13)

In [8]:
df.head(2)

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,Industry,Sector,Revenue
1,1,Entry Level Data Conversion Analyst,$35K - $71K (Glassdoor est.),"As the Data Conversion Analyst, you will be co...",3.4,Reynolds and Reynolds\n3.4,"Dayton, OH",1001 to 5000 Employees,1866,Company - Private,Computer Hardware & Software,Information Technology,$2 to $5 billion (USD)
3,3,Senior Data Analyst with Visualization (No spo...,$63K - $149K (Glassdoor est.),Senior Data Analyst with Visualization (No spo...,3.7,HCL Technologies\n3.7,"Burlingame, CA",10000+ Employees,1991,Company - Public,Computer Hardware & Software,Information Technology,$5 to $10 billion (USD)


In [9]:
# Fixed Salary 

df['Hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)
df['Employer provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary' in x.lower() else 0)

fixed_salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])
Minus_kd = fixed_salary.apply(lambda x: x.replace('K','').replace('$',''))
Minus_hour_provided = Minus_kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))

df['Min Salary'] = Minus_hour_provided.apply(lambda x: int(x.split('-')[0]))
df['Max Salary'] = Minus_hour_provided.apply(lambda x: int(x.split('-')[1]))
df['Avg Salary'] = (df['Min Salary']+df['Max Salary'])/2



In [10]:
# Fixed Company

df['Company_text'] = df['Company Name'].apply(lambda x: x.split('\n')[0])
print(df['Company_text'].head(2))

1    Reynolds and Reynolds
3         HCL Technologies
Name: Company_text, dtype: object


In [11]:
# Adding features (State, Age)

df['State'] = df['Location'].apply(lambda x: x.split(',')[-1])
df['Age'] = df['Founded'].apply(lambda x: x if int(x)<1 else 2022-int(x))



In [16]:
# Adding other features

df['Python'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
df['Tableau'] = df['Job Description'].apply(lambda x: 1 if 'tableau' in x.lower() else 0)
df['Excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df['Power BI'] = df['Job Description'].apply(lambda x: 1 if 'power bi' in x.lower() else 0)
df['SAS'] = df['Job Description'].apply(lambda x: 1 if 'sas' in x.lower() else 0)
df['SQL'] = df['Job Description'].apply(lambda x: 1 if 'sql' in x.lower() else 0)
df['SSIS'] = df['Job Description'].apply(lambda x: 1 if 'ssis' in x.lower() else 0)


In [17]:
df.head(2)

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Size,Founded,Type of ownership,...,Company_text,State,Age,Python,Tableau,Excel,Power BI,SAS,SQL,SSIS
1,1,Entry Level Data Conversion Analyst,$35K - $71K (Glassdoor est.),"As the Data Conversion Analyst, you will be co...",3.4,Reynolds and Reynolds\n3.4,"Dayton, OH",1001 to 5000 Employees,1866,Company - Private,...,Reynolds and Reynolds,OH,156,0,0,0,0,0,0,0
3,3,Senior Data Analyst with Visualization (No spo...,$63K - $149K (Glassdoor est.),Senior Data Analyst with Visualization (No spo...,3.7,HCL Technologies\n3.7,"Burlingame, CA",10000+ Employees,1991,Company - Public,...,HCL Technologies,CA,31,0,0,0,0,0,0,0


In [19]:
df.columns

Index(['index', 'Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Size', 'Founded', 'Type of ownership',
       'Industry', 'Sector', 'Revenue', 'Hourly', 'Employer provided',
       'Min Salary', 'Max Salary', 'Avg Salary', 'Company_text', 'State',
       'Age', 'Python', 'Tableau', 'Excel', 'Power BI', 'SAS', 'SQL', 'SSIS'],
      dtype='object')

In [23]:
df_output = df

df_output.to_csv('da_salary_cleaned', index = False)
